In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_excel("Dataa.xlsx")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['Email ID'] = df['Email ID'].str.strip().str.lower()
df['Events'] = df['Events'].str.strip().str.lower()

In [ ]:
# function to resolve conflicts in data
def resolve_conflicts(series):
    if pd.api.types.is_numeric_dtype(series) and series.name == "CGPA":
        return series.median()  # Use median for Continuous data like cgpa and mode for discrete data
    else:
        mode = series.mode()
        return mode.iloc[0] if not mode.empty else series.iloc[0]

In [ ]:
#Group by Emaild Id and Events to resolve the conflict
df_cleaned = df.groupby(['Email ID', 'Events']).agg(resolve_conflicts).reset_index()

In [ ]:
#mode conflict flag per student
def mode_or_first(series):
    mode = series.mode()
    if len(mode) == 1:
        return mode.iloc[0]
    return mode.iloc[0] if not mode.empty else series.iloc[0]

def mode_conflict(series):
    return len(series.mode()) > 1

In [ ]:
# Apply both: values and conflict flags
student_profile = df.groupby("Email ID").agg({
    "Year of Graduation": mode_or_first,
    "Experience with python (Months)": mode_or_first,
    "College Name": mode_or_first
}).reset_index()

conflict_flags = df.groupby("Email ID").agg({
    "Year of Graduation": mode_conflict,
    "Experience with python (Months)": mode_conflict,
    "College Name": mode_conflict
}).rename(columns={
    "Year of Graduation": "Grad_Conflict",
    "Experience with python (Months)": "PythonExp_Conflict",
    "College Name": "College_Conflict"
}).reset_index()

In [ ]:
# Merge conflict flags into profile
student_profile_full = student_profile.merge(conflict_flags, on="Email ID", how="left")

In [ ]:
#  Drop student-level fields from event-level cleaned data
df_cleaned = df_cleaned.drop(columns=[
    "Year of Graduation",
    "Experience with python (Months)",
    "College Name"
], errors="ignore")

df_final = df_cleaned.merge(student_profile_full, on="Email ID", how="left")

In [ ]:
# Group by Email ID to check CGPA range and unique cities
integrity_check = df_cleaned.groupby("Email ID").agg({
    "CGPA": lambda x: x.max() - x.min(),
    "City": pd.Series.nunique
}).rename(columns={
    "CGPA": "CGPA_Range",
    "City": "Unique_Cities"
}).reset_index()

In [ ]:
# Set thresholds
CGPA_THRESHOLD = 1.0
CITY_THRESHOLD = 1

In [ ]:
# Add conflict flags
integrity_check["CGPA_Conflict"] = integrity_check["CGPA_Range"] > CGPA_THRESHOLD
integrity_check["City_Conflict"] = integrity_check["Unique_Cities"] > CITY_THRESHOLD


In [ ]:
# Add trustworthiness flag
integrity_check["Is_Trusted"] = ~(integrity_check["CGPA_Conflict"] | integrity_check["City_Conflict"])

In [ ]:
df_final = df_final.merge(integrity_check[["Email ID", "Is_Trusted", "CGPA_Conflict", "City_Conflict"]], on="Email ID", how="left")

In [ ]:
# Save cleaned and flagged data
df_final.to_excel("Internship.xlsx", index=False)

# Save integrity report separately
integrity_check.to_excel("Student_Data_Integrity_Report.xlsx", index=False)

print("✅ Cleaned and flagged data saved as 'Fully_Cleaned_Student_Data.xlsx'")
print("📄 Integrity report saved as 'Student_Data_Integrity_Report.xlsx'")



✅ Cleaned and flagged data saved as 'Fully_Cleaned_Student_Data.xlsx'
📄 Integrity report saved as 'Student_Data_Integrity_Report.xlsx'
